In [1]:
import dataverk as dv
from dataverk_tools.themes import nav

import pandas as pd
import ipywidgets as widgets
import requests
from io import StringIO

import json
import plotly.graph_objs as go
import plotly.io as plio
import plotly_express as px
import plotly.figure_factory as ff

from datetime import datetime
from keplergl import KeplerGl 

# Henter tall for arbeidsledige fra SSB

In [2]:
query = {
  "query": [
    {
      "code": "Region",
      "selection": {
        "filter": "all",
        "values": [
          "*"
        ]
      }
    },
    {
      "code": "Kjonn",
      "selection": {
        "filter": "item",
        "values": [
          "0"
        ]
      }
    },
    {
      "code": "ContentsCode",
      "selection": {
        "filter": "item",
        "values": [
          "Registrerte1"
        ]
      }
    },
    {
      "code": "Tid",
      "selection": {
        "filter": "item",
        "values": [
          "2018M11"
        ]
      }
    }
  ],
  "response": {
    "format": "csv"
  }
}


url = "https://data.ssb.no/api/v0/no/table/10594"

In [3]:
r = requests.post(url, data=json.dumps(query))

In [4]:
df = pd.read_csv(StringIO(r.text))

In [5]:
df["region_id"] = df["region"].apply(lambda x: x.split(" ")[0])
df["region_navn"] = df["region"].apply(lambda x: x.split(" ")[1])

In [6]:
df_ledige = df[(df["region_id"].str.len()==4) & (df["kjønn"]=="Begge kjønn")].drop(columns=['region', 'kjønn'])
df_ledige.columns=["ledige","kommunenummer", "kommune"]
df_ledige.drop_duplicates(subset="kommunenummer", keep='last', inplace=True)
df_ledige.head()

,ledige,kommunenummer,kommune
2,387,0101,Halden
3,.,0102,Sarpsborg
4,.,0103,Fredrikstad
5,491,0104,Moss
6,833,0105,Sarpsborg


# Henter kartdata

In [7]:
url = 'https://raw.githubusercontent.com/datasett/maps/master/norway/data/counties_2018_s_geojson.json'
df_geo = pd.read_json(url)

In [8]:
def getData(row):
    return row['properties']['centroid_kommune']

In [9]:
df_geo['centroid_kommune'] = df_geo['features'].apply(getData)

In [10]:
df_geo.head(1)['features'][0]

{'type': 'Feature',
 'geometry': {'type': 'Polygon',
  'coordinates': [[[11.557944804290678, 59.25961243805086],
    [11.71052391528105, 59.0336797054564],
    [11.65200300120283, 58.906233335204945],
    [11.456861418997182, 58.890755310196596],
    [11.47061404327044, 58.99288736211179],
    [11.374472843176518, 59.10382339848626],
    [11.198454406864782, 59.09297294006742],
    [11.231868129077334, 59.135005904455014],
    [11.342908158671758, 59.25830981486017],
    [11.557944804290678, 59.25961243805086]]]},
 'properties': {'kommunenummer': '0101',
  'oppdateringsdato': '2018-12-10T00:00:00',
  'navn': 'Halden',
  'area_kommune': 0.10001455200841401,
  'centroid_kommune': [11.498404769493327, 59.08595732263681],
  'fylke_nummer_2018': '01',
  'fylke_nummer_2019': '01',
  'fylke_navn_2019': 'ØSTFOLD',
  'fylke_nummer_2020': '30',
  'fylke_navn_2020': 'VIKEN'},
 'id': '0'}

In [17]:
def getId(row):
    return row['features']['properties']['kommunenummer']
    
def getGeom(row):
    return json.dumps(row['features']['geometry'])

In [18]:
df_geo['id'] = df_geo.apply(getId, axis=1)
df_geo['kommunenummer'] = df_geo.apply(getId, axis=1)
df_geo['geometry_string'] = df_geo.apply(getGeom, axis=1)
df_geo.head(1)

,type,features,centroid_kommune,id,kommunenummer,geometry_string
0,FeatureCollection,"{'type': 'Feature', 'geometry': {'type': 'Poly...","[11.498404769493327, 59.08595732263681]",0101,0101,"{""type"": ""Polygon"", ""coordinates"": [[[11.55794..."


In [19]:
dt = pd.merge(df_geo, df_ledige, on="kommunenummer")

In [20]:
dt.head()

,type,features,centroid_kommune,id,kommunenummer,geometry_string,ledige,kommune
0,FeatureCollection,"{'type': 'Feature', 'geometry': {'type': 'Poly...","[11.498404769493327, 59.08595732263681]",0101,0101,"{""type"": ""Polygon"", ""coordinates"": [[[11.55794...",387,Halden
1,FeatureCollection,"{'type': 'Feature', 'geometry': {'type': 'Poly...","[10.694785483673224, 59.45187621919044]",0104,0104,"{""type"": ""Polygon"", ""coordinates"": [[[10.68526...",491,Moss
2,FeatureCollection,"{'type': 'Feature', 'geometry': {'type': 'Poly...","[11.161866574429215, 59.288044170917]",0105,0105,"{""type"": ""Polygon"", ""coordinates"": [[[11.13771...",833,Sarpsborg
3,FeatureCollection,"{'type': 'Feature', 'geometry': {'type': 'Poly...","[10.937506651630564, 59.23057820000602]",0106,0106,"{""type"": ""Polygon"", ""coordinates"": [[[11.08331...",1236,Fredrikstad
4,FeatureCollection,"{'type': 'Feature', 'geometry': {'type': 'Poly...","[10.965155146323387, 59.07160447267584]",0111,0111,"{""type"": ""Polygon"", ""coordinates"": [[[11.04347...",41,Hvaler


In [21]:
df_with_geometry = dt[['id','kommune','ledige','geometry_string']]

# add to map
from keplergl import KeplerGl
map_1 = KeplerGl()
map_1.add_data(df_with_geometry, "kommuner")
map_1

User Guide: https://github.com/keplergl/kepler.gl/blob/master/docs/keplergl-jupyter/user-guide.md


KeplerGl(data={'kommuner': {'index': [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20…

# Example

In [22]:
df_centroid_kommune = dt[['id','kommune','ledige','centroid_kommune']]
df_centroid_kommune['lat'] = df_centroid_kommune['centroid_kommune'].apply(lambda x: x[1])
df_centroid_kommune['lng'] = df_centroid_kommune['centroid_kommune'].apply(lambda x: x[0])

In [23]:
df_centroid_kommune.head()

,id,kommune,ledige,centroid_kommune,lat,lng
0,0101,Halden,387,"[11.498404769493327, 59.08595732263681]",59.085957,11.498405
1,0104,Moss,491,"[10.694785483673224, 59.45187621919044]",59.451876,10.694785
2,0105,Sarpsborg,833,"[11.161866574429215, 59.288044170917]",59.288044,11.161867
3,0106,Fredrikstad,1236,"[10.937506651630564, 59.23057820000602]",59.230578,10.937507
4,0111,Hvaler,41,"[10.965155146323387, 59.07160447267584]",59.071604,10.965155


In [31]:
config = {
    "version": "v1",
    "config": {
        "visState": {
            "layers": [
                {
                    "type": "point",
                    "config": {
                        "dataId": "kommune",
                        "isVisible": True,
                        "columns": {
                            "altitude": None,
                            "lat": "lat",
                            "lng": "lng"
                        },
                        "color": [165, 16, 55],
                        "visConfig": {
                            "radius": 10
                        },
                        "textLabel": {
                            "field": {"name": "kommune", "type": "string"}
                        }
                    }
                }
            ],
            "interactionConfig": {
                "tooltip": {
                    "fieldsToShow": {
                        "kommune": ["kommune"]
                    },
                    "enabled": True
                }
            }
            
        }
    }
}

In [45]:
hex_data = "https://raw.githubusercontent.com/keplergl/kepler.gl/master/bindings/kepler.gl-jupyter/notebooks/hex-data.csv"
df_hex = pd.read_csv(hex_data)
df_hex.head()

,hex_id,value,is_true,float_value,empty,time
0,89283082c2fffff,64,True,64.1,NaN,11/1/17 11:00
1,8928308288fffff,73,True,73.1,NaN,11/1/17 11:00
2,89283082c07ffff,65,True,65.1,NaN,11/1/17 11:00
3,89283082817ffff,74,True,74.1,NaN,11/1/17 11:00
4,89283082c3bffff,66,True,66.1,NaN,11/1/17 11:00


In [37]:
hex_config = {u'version': u'v1', u'config': {u'visState': {u'layers': [{u'type': u'hexagonId', u'visualChannels': {u'sizeField': {u'type': u'integer', u'name': u'value'}, u'coverageField': None, u'colorScale': u'quantize', u'coverageScale': u'linear', u'colorField': {u'type': u'integer', u'name': u'value'}, u'sizeScale': u'linear'}, u'config': {u'dataId': u'data_1', u'color': [250, 116, 0], u'textLabel': {u'color': [255, 255, 255], u'field': None, u'size': 50, u'anchor': u'middle', u'offset': [0, 0]}, u'label': u'H3 Hexagon', u'isVisible': True, u'visConfig': {u'coverageRange': [0, 1], u'opacity': 0.8, u'elevationScale': 5, u'hi-precision': False, u'coverage': 1, u'enable3d': True, u'sizeRange': [0, 500], u'colorRange': {u'category': u'Uber', u'type': u'sequential', u'colors': [u'#194266', u'#355C7D', u'#63617F', u'#916681', u'#C06C84', u'#D28389', u'#E59A8F', u'#F8B195'], u'reversed': False, u'name': u'Sunrise 8'}}, u'columns': {u'hex_id': u'hex_id'}}, u'id': u'jdys7lp'}], u'interactionConfig': {u'brush': {u'enabled': False, u'size': 0.5}, u'tooltip': {u'fieldsToShow': {u'data_1': [u'hex_id', u'value']}, u'enabled': True}}, u'splitMaps': [], u'layerBlending': u'normal', u'filters': []}, u'mapState': {u'bearing': 2.6192893401015205, u'dragRotate': True, u'zoom': 12.32053899007826, u'longitude': -122.42590232651203, u'isSplit': False, u'pitch': 37.374216241015446, u'latitude': 37.76209132041332}, u'mapStyle': {u'mapStyles': {}, u'topLayerGroups': {}, u'styleType': u'dark', u'visibleLayerGroups': {u'building': True, u'land': True, u'3d building': False, u'label': True, u'water': True, u'border': False, u'road': True}}}}

In [44]:
sf_json = "https://github.com/keplergl/kepler.gl/raw/master/bindings/kepler.gl-jupyter/notebooks/sf_zip_geo.json"
r = requests.get(sf_json)
json_data = r.json()

In [50]:
w1 = keplergl.KeplerGl(height=500)
w1.add_data(df_hex, 'data_1')
w1.add_data(json_data, 'geojson')
w1.config = hex_config
w1

User Guide: https://github.com/keplergl/kepler.gl/blob/master/docs/keplergl-jupyter/user-guide.md


KeplerGl(config={'version': 'v1', 'config': {'visState': {'layers': [{'type': 'hexagonId', 'visualChannels': {…

In [52]:
fylker_2020 = "http://eric.clst.org/assets/wiki/uploads/Stuff/gz_2010_us_040_00_500k.json"
country_gdf = geopandas.read_file(url)
display(country_gdf.head(5))

  Complete output from command /Users/B149899/anaconda3/envs/dv/bin/python -u -c "import setuptools, tokenize;__file__='/private/var/folders/h9/b04cdyzx2c1bt2y9c05349yr0000gn/T/pip-install-_99kc_5d/h3/setup.py';f=getattr(tokenize, 'open', open)(__file__);code=f.read().replace('\r\n', '\n');f.close();exec(compile(code, __file__, 'exec'))" bdist_wheel -d /private/var/folders/h9/b04cdyzx2c1bt2y9c05349yr0000gn/T/pip-wheel-zfpxmzrd --python-tag cp37:
  running bdist_wheel
  running build
  running build_py
  creating build
  creating build/lib.macosx-10.7-x86_64-3.7
  creating build/lib.macosx-10.7-x86_64-3.7/h3
  copying h3/h3.py -> build/lib.macosx-10.7-x86_64-3.7/h3
  copying h3/__init__.py -> build/lib.macosx-10.7-x86_64-3.7/h3
  running build_ext
  + VERSION=v3.4.2
  + IS_64BITS=True
  + '[' '' == v3.4.2 ']'
  + command -v cmake
  + echo 'cmake required but not found.'
  cmake required but not found.
  + exit 1
  Traceback (most recent call last):
    File "<string>", line 1, in <modul

In [51]:
w1.config

{'version': 'v1',
 'config': {'visState': {'filters': [],
   'layers': [{'id': 'jdys7lp',
     'type': 'hexagonId',
     'config': {'dataId': 'data_1',
      'label': 'H3 Hexagon',
      'color': [250, 116, 0],
      'columns': {'hex_id': 'hex_id'},
      'isVisible': True,
      'visConfig': {'opacity': 0.8,
       'colorRange': {'category': 'Uber',
        'type': 'sequential',
        'colors': ['#194266',
         '#355C7D',
         '#63617F',
         '#916681',
         '#C06C84',
         '#D28389',
         '#E59A8F',
         '#F8B195'],
        'reversed': False,
        'name': 'Sunrise 8'},
       'coverage': 1,
       'sizeRange': [0, 500],
       'coverageRange': [0, 1],
       'elevationScale': 5},
      'textLabel': [{'field': None,
        'color': [255, 255, 255],
        'size': 50,
        'offset': [0, 0],
        'anchor': 'middle',
        'alignment': 'center'}]},
     'visualChannels': {'colorField': {'name': 'value', 'type': 'integer'},
      'colorScale': 'q

In [32]:
import keplergl
w1 = keplergl.KeplerGl(height=500)
w1.add_data(data=df_centroid_kommune, name="kommune")
w1.config=config
w1

User Guide: https://github.com/keplergl/kepler.gl/blob/master/docs/keplergl-jupyter/user-guide.md


KeplerGl(config={'version': 'v1', 'config': {'visState': {'layers': [{'type': 'point', 'config': {'dataId': 'k…

In [28]:
w1.config

{'version': 'v1',
 'config': {'visState': {'filters': [],
   'layers': [{'id': 'je1tz4m',
     'type': 'point',
     'config': {'dataId': 'kommune',
      'label': 'new layer',
      'color': [165, 16, 55],
      'columns': {'lat': 'lat', 'lng': 'lng', 'altitude': None},
      'isVisible': True,
      'visConfig': {'radius': 10,
       'fixedRadius': False,
       'opacity': 0.8,
       'outline': False,
       'thickness': 2,
       'strokeColor': [165, 16, 55],
       'colorRange': {'name': 'Global Warming',
        'type': 'sequential',
        'category': 'Uber',
        'colors': ['#5A1846',
         '#900C3F',
         '#C70039',
         '#E3611C',
         '#F1920E',
         '#FFC300']},
       'strokeColorRange': {'name': 'Global Warming',
        'type': 'sequential',
        'category': 'Uber',
        'colors': ['#5A1846',
         '#900C3F',
         '#C70039',
         '#E3611C',
         '#F1920E',
         '#FFC300']},
       'radiusRange': [0, 50],
       'filled': Tr

# Create datapackage

In [143]:
readme = """
# Eksempler på kartvisninger med webGL: Arbeidsledige
"""

metadata = {
'store': 'gs',
'repo': 'https://raw.githubusercontent.com/datasett/nav-examples/master/',
'readme': readme,
'license':'MIT',
'accessRights':'Open',
'author': 'paul.bencze@nav.no',
'title':'Eksempler på kart med webGL: Arbeidsledige',
'title':'Eksempler på kart med webGL: Arbeidsledige',
'description':'Eksempler på kartvisninger med webGL: Arbeidsledige',
'name':'Eksempler på kart med webGL: Arbeidsledige',
'source':'NAV',
'keywords':['kart, eksempler', 'examples', 'arbeidsledige'],
'provenance':'NAV',
'publisher': 'NAV',
'bucket': 'nav-opendata',
}

In [145]:
dp = dv.Datapackage(metadata)

In [155]:
data = df_centroid_kommune.to_json(orient='records')

In [156]:
spec = {
    "data": data,
    "map": "https://raw.githubusercontent.com/datasett/maps/master/norway/data/counties_2018_s_geojson.json",
    "colorby": "ledige",
    "value": 'ledige',
    "lineColor": [200,200,200], 
    "initialViewState": {
      "longitude": 14,
      "latitude": 65,
      "zoom": 3.5,
      "pitch": 0,
      "bearing": 0,
    }
  }

In [157]:
description = '''
Kart beskrivelse
'''

dp.add_view(
    spec_type='map',
    name='Kommuner farge etter antall ledige',
    description = description,
    resources=[],
    attribution = "Kilde: NAV.",
    spec=spec
)

In [158]:
import os
os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = '/users/b149899/projects/binder/binder/keys/nav_opendata.json'

In [159]:
client = dv.Client()
client.publish(dp)

gs


TypeError: Object of type DataFrame is not JSON serializable